# 1. 라이브러리 설치 및 폴더 생성
 프로젝트에 필요한 라이브러리인 `konlpy`와 `japanize_kivy`를 설치하고, 이후 텍스트 파일을 저장할 `tmp` 폴더를 생성합니다. `os`와 `re` 모듈은 파일 및 디렉토리 작업을, `Path`는 경로 관리를, `Okt`는 한글 텍스트 처리를 위해 사용됩니다
넘어갑니다.
경로에 생성합니다.
경로에 생성합니다.
로에 생성합니다.
다.


In [9]:
!pip install konlpy japanize_kivy

In [10]:
import os
import re
from pathlib import Path
from konlpy.tag import Okt
os.makedirs("tmp", exist_ok=True)

In [11]:
# 구글 드라이브로 연결 (lyrics.zip 불러오기를 위해)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# 가사 파일 Unzip
!unzip -q "/content/drive/MyDrive/Python Code/lyrics.zip" -d "/content/drive/MyDrive/Python Code/lyrics"

# 2. 가사 파일 읽기 및 인덱스 파일 생성
 `lyrics` 폴더 내의 모든 `lyrics_*.txt` 파일을 읽어 각 가사 블럭(일본어/한글발음/한글번역)을 `index.txt` 파일에 인덱스를 부여하여 저장합니다. 각 가사 블럭은 인덱스를 포함하여 `index.txt` 파일에 순차적으로 저장됩니다. 최종적으로 저장된 가사 블럭의 수를 출력합니다.
.


In [14]:
lyrics_folder = Path('/content/drive/MyDrive/Python Code/lyrics')

index_file = Path('tmp/index.txt')

index = 1

with index_file.open('w', encoding='utf-8') as index_f:
    for lyrics_file in sorted(lyrics_folder.glob('lyrics_*.txt')):
        with lyrics_file.open('r', encoding='utf-8') as f:
            lines = f.readlines()

            for i in range(0, len(lines), 3):
                if i + 2 < len(lines):
                    jp_line = lines[i].strip()
                    kr_pron_line = lines[i+1].strip()
                    kr_trans_line = lines[i+2].strip()

                    index_f.write(f"{index}\n{jp_line}\n{kr_pron_line}\n{kr_trans_line}\n")
                    index += 1

print(f"'index.txt' 파일에 {index-1}개의 가사 블럭이 저장되었습니다.")

'index.txt' 파일에 190704개의 가사 블럭이 저장되었습니다.


# 3. 가사 블럭 전처리 및 유효성 검증
이 셀에서는 `index.txt` 파일에서 각 가사 블럭을 읽어와 유효한지 검증합니다.
`invalid_detect` 함수를 사용하여 한글 발음 라인의 유효성을 검사하고, 유효하지 않은 블럭은 제외합니다.
유효성 검사의 기준은 다음과 같습니다:
1. 한글 발음 라인에 문장부호를 제외한 특수포함된가 있는 경우
2. 한글 발음 라인에포함된어가 있는 경우
3. 한글 발음 라인에 유효하지 않은 모음이 포함된 경우
4. 한글 발음 라인에 유효하지 않은 받침이 포함된 경우
5. 한글완전하지 못한 음절이 포함된 경우
6. 일본어 곡의 가사가 아닌 경우 존재하는 경우

유효한 가사 블럭의 인덱스를 `valid_indices`에 저장하고, 유효하지 않은 가사 블럭의 인덱스를 `invalid_indices`에 저장합니다.
최종적으로 유효한 가사 블럭의 수와 유효하지 않은 가사 블럭 수를 출력합니다.
.


In [15]:
def decompose_hangul(char):
    start_unicode = 0xAC00
    end_unicode = 0xD7A3

    chosung = ["ㄱ", "ㄲ", "ㄴ", "ㄷ", "ㄸ", "ㄹ", "ㅁ", "ㅂ", "ㅃ", "ㅅ",
               "ㅆ", "ㅇ", "ㅈ", "ㅉ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ"]

    jungsung = ["ㅏ", "ㅐ", "ㅑ", "ㅒ", "ㅓ", "ㅔ", "ㅕ", "ㅖ", "ㅗ", "ㅘ",
                "ㅙ", "ㅚ", "ㅛ", "ㅜ", "ㅝ", "ㅞ", "ㅟ", "ㅠ", "ㅡ", "ㅢ", "ㅣ"]

    jongsung = ["", "ㄱ", "ㄲ", "ㄳ", "ㄴ", "ㄵ", "ㄶ", "ㄷ", "ㄹ", "ㄺ", "ㄻ",
                "ㄼ", "ㄽ", "ㄾ", "ㄿ", "ㅀ", "ㅁ", "ㅂ", "ㅄ", "ㅅ", "ㅆ",
                "ㅇ", "ㅈ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ"]

    char_code = ord(char)

    if start_unicode <= char_code <= end_unicode:
        index = char_code - start_unicode

        chosung_index = index // (21 * 28)
        jungsung_index = (index // 28) % 21
        jongsung_index = index % 28

        return (chosung[chosung_index], jungsung[jungsung_index], jongsung[jongsung_index])
    else:
        raise ValueError("입력된 문자는 한글 음절이 아닙니다.")

def invalid_detect(block):
    jp_line, _, kr_pron_line, _ = block

    if re.search(r'[^ㄱ-ㅎ가-힣\s.,?!-]', kr_pron_line):
        return True

    if re.search(r'[a-zA-Z]', kr_pron_line):
        return True

    invalid_vowels = {'ㅓ', 'ㅕ', 'ㅐ', 'ㅒ', 'ㅖ', 'ㅙ', 'ㅚ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅢ'}
    for char in kr_pron_line:
        if '가' <= char <= '힣':
            _, medial, _ = decompose_hangul(char)
            if medial in invalid_vowels:
                return True

    invalid_final_consonants = {'ㄹ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㄲ', 'ㅆ', 'ㄳ', 'ㄵ', 'ㄶ', 'ㄺ', 'ㄻ', 'ㄽ', 'ㄼ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅄ'}
    for char in kr_pron_line:
        if '가' <= char <= '힣':
            _, _, final = decompose_hangul(char)
            if final in invalid_final_consonants:
                return True

    if re.search(r'[ㄱ-ㅎㅏ-ㅣ]', kr_pron_line):
        return True

    if re.search(r'[가-힣]', jp_line):
        return True

    return False

index_file = Path('tmp/index.txt')

valid_indices = []
invalid_indices = []

invalid_count = 0

with index_file.open('r', encoding='utf-8') as index_f:
    lines = index_f.readlines()

    for i in range(0, len(lines), 4):
        if i + 3 < len(lines):
            block = lines[i:i+4]
            index = block[0].strip()
            jp_line = block[1].strip()
            kr_pron_line = block[2].strip()
            kr_trans_line = block[3].strip()

            if invalid_detect((jp_line, index, kr_pron_line, kr_trans_line)):
                invalid_count += 1
                invalid_indices.append(index)
            else:
                valid_indices.append(index)

print(f"유효한 가사 블럭의 수: {len(valid_indices)}")
print(f"유효하지 않은 가사 블럭의 수: {invalid_count}")

유효한 가사 블럭의 수: 169697
유효하지 않은 가사 블럭의 수: 21007


# 4. 한글 발음 전처리 및 저장
이 셀에서는 유효한 가사 블럭의 한글 발음 라인에 대해 전처리를 수행합니다. `preprocess` 함수를 사용하여 특수문자, 띄어쓰기, 문장 부호를 모두 삭제하고 오로지 한글만 남깁니다. 전처리된 한글 발음 라인은 `preprocessed.txt` 파일에 인덱스와 함께 저장됩니다.
- `preprocess` 함수는 한글만 남기는 전처리를 수행하고, 한글 외의 문자가 포함된 라인은 제외합니다.
- 유효한 가사 블럭의 인덱스는 `valid_indices_set`에 저장되어 있어 이를 기준으로 전처리할 블럭을 식별합니다.

최종적으로 `preprocessed.txt` 파일에 전처리가 완료된 유효한 한글 발음 라인이 인덱스와 함께 저장됩니다.


In [16]:
def preprocess(kr_pron_line):
    kr_pron_line = re.sub(r'[^ㄱ-ㅎ가-힣]', '', kr_pron_line)

    if all('가' <= char <= '힣' or 'ㄱ' <= char <= 'ㅎ' for char in kr_pron_line):
        return kr_pron_line
    return None

preprocessed_file = Path('tmp/preprocessed.txt')

with preprocessed_file.open('w', encoding='utf-8') as preprocessed_f:
    with index_file.open('r', encoding='utf-8') as index_f:
        lines = index_f.readlines()

        valid_indices_set = set(valid_indices)

        for i in range(0, len(lines), 4):
            if i + 3 < len(lines):
                block = lines[i:i+4]
                index = block[0].strip()
                kr_pron_line = block[2].strip()

                if index in valid_indices_set:
                    processed_line = preprocess(kr_pron_line)
                    if processed_line:
                        preprocessed_f.write(f"{index}\t{processed_line}\n")

print(f"'preprocessed.txt' 파일에 유효한 가사 블럭들이 저장되었습니다.")

'preprocessed.txt' 파일에 유효한 가사 블럭들이 저장되었습니다.


# 5. 한글 발음의 모음을 히라가나로 추출 및 저장
이 셀에서는 전처리된 한글 발음 라인에서 모음을 히라가나로 변환하여 `vowel.txt` 파일에 저장합니다.
- `vowel_extraction` 함수는 각 한글 음절을 분해하여 모음을 히라가나로 치환하고, 받침이 있는 경우 이를 적절한 히라가나로 추가합니다.
- 변환된 히라가나 모음 문자열은 인덱스와 함께 `vowel.txt` 파일에 저장됩니다.

최종적으로 `vowel.txt` 파일에 각 인덱스와 대응하는 히라가나 모음 문자열이 저됩니다.


In [17]:
def vowel_extraction(kr_pron_line):
    hiragana_map = {
        'ㅏ': 'あ', 'ㅑ': 'あ', 'ㅘ': 'あ',
        'ㅣ': 'い',
        'ㅜ': 'う', 'ㅡ': 'う', 'ㅠ': 'う',
        'ㅔ': 'え',
        'ㅗ': 'お', 'ㅛ': 'お'
    }
    final_consonant_map = {
        'ㄱ': 'っ', 'ㄷ': 'っ', 'ㅂ': 'っ', 'ㅅ': 'っ',
        'ㄴ': 'ん', 'ㅁ': 'ん', 'ㅇ': 'ん'
    }
    vowels = []

    for char in kr_pron_line:
        if '가' <= char <= '힣':
            initial, medial, final = decompose_hangul(char)

            if medial in hiragana_map:
                vowels.append(hiragana_map[medial])

            if final in final_consonant_map:
                vowels[-1] += final_consonant_map[final]

    return ''.join(vowels)

vowel_file = Path('tmp/vowel.txt')

with vowel_file.open('w', encoding='utf-8') as vowel_f:
    with preprocessed_file.open('r', encoding='utf-8') as preprocessed_f:
        lines = preprocessed_f.readlines()

        for line in lines:
            index, kr_pron_line = line.strip().split('\t')
            vowels = vowel_extraction(kr_pron_line)
            vowel_f.write(f"{index} : {vowels}\n")

print(f"'vowel.txt' 파일에 모음이 히라가나로 변환되어 저장되었습니다.")

'vowel.txt' 파일에 모음이 히라가나로 변환되어 저장되었습니다.


# 6. 사용자 입력 처리 및 라임 찾기
이 셀에서는 사용자의 입력을 받아 유효성을 검증하고, 라임이 일치하는 구절을 찾습니다.

1. **`convert_hiragana_vowel` 함수**:
    - 히라가나 문자열을 받아서 대응되는 모음으로 변환합니다.

2. **`process_input` 함수**:
    - 사용자의 입력이 한글일 경우:
        1. `invalid_detect` 함수를 사용하여 유효성을 검증합니다.
        2. `preprocess` 함수를 통해 전처리를 수행한 후 `vowel_extraction` 함수를 통해 히라가나 모음으로 변환합니다.
    - 사용자의 입력이 히라가나일 경우:
        1. `convert_hiragana_vowel` 함수를 사용하여 같은 단의 모음으로 변환합니다.
    - 유효하지 않은 입력일 경우 예외를 발생시킵니다.

3. **`find_matching_indices` 함수**:
    - `vowel.txt` 파일에서 입력 받은 히라가나 모음 문자열과 일치하는 인덱스를 검색합니다.
    - 라임이 끝부분에 일치하는 구절은 우선 순위를 부여합니다.
    - 첫 번째 우선 순위(끝부분이 일치하는 구절)와 두 번째 우선 순위(중간 또는 앞부분에 일치하는 구절)를 구분하여 인덱스를 리스트에 저장합니다.

4. 사용자 입력을 받아 라임을 찾고, 결과를 출력합니다:
    - 유효한 입력일 경우, 라임이 일치하는 구절의 총 개수와 끝부분이 일치하는 구절의 개수를 출력합니다.
    - 유효하지 않은 입력일 경우, 예외 메시지를 출력합니다.


In [18]:
def convert_hiragana_vowel(hiragana):
    hiragana_vowel_map = {
        'あ': 'あ', 'い': 'い', 'う': 'う', 'え': 'え', 'お': 'お',
        'か': 'あ', 'き': 'い', 'く': 'う', 'け': 'え', 'こ': 'お',
        'が': 'あ', 'ぎ': 'い', 'ぐ': 'う', 'げ': 'え', 'ご': 'お',
        'さ': 'あ', 'し': 'い', 'す': 'う', 'せ': 'え', 'そ': 'お',
        'ざ': 'あ', 'じ': 'い', 'ず': 'う', 'ぜ': 'え', 'ぞ': 'お',
        'た': 'あ', 'ち': 'い', 'つ': 'う', 'て': 'え', 'と': 'お',
        'だ': 'あ', 'ぢ': 'い', 'づ': 'う', 'で': 'え', 'ど': 'お',
        'な': 'あ', 'に': 'い', 'ぬ': 'う', 'ね': 'え', 'の': 'お',
        'は': 'あ', 'ひ': 'い', 'ふ': 'う', 'へ': 'え', 'ほ': 'お',
        'ば': 'あ', 'び': 'い', 'ぶ': 'う', 'べ': 'え', 'ぼ': 'お',
        'ぱ': 'あ', 'ぴ': 'い', 'ぷ': 'う', 'ぺ': 'え', 'ぽ': 'お',
        'ま': 'あ', 'み': 'い', 'む': 'う', 'め': 'え', 'も': 'お',
        'や': 'あ', 'ゆ': 'う', 'よ': 'お',
        'ら': 'あ', 'り': 'い', 'る': 'う', 'れ': 'え', 'ろ': 'お',
        'わ': 'あ', 'を': 'お', 'ん': 'ん',
        'きゃ': 'あ', 'しゃ': 'あ', 'ちゃ': 'あ', 'にゃ': 'あ', 'ひゃ': 'あ', 'みゃ': 'あ', 'りゃ': 'あ',
        'きゅ': 'う', 'しゅ': 'う', 'ちゅ': 'う', 'にゅ': 'う', 'ひゅ': 'う', 'みゅ': 'う', 'りゅ': 'う',
        'きょ': 'お', 'しょ': 'お', 'ちょ': 'お', 'にょ': 'お', 'ひょ': 'お', 'みょ': 'お', 'りょ': 'お',
        'っ': 'っ', 'ゃ': '', 'ゅ': '', 'ょ': ''
    }
    result = []
    i = 0
    while i < len(hiragana):
        char = hiragana[i]
        if char in ('ゃ', 'ゅ', 'ょ') and i > 0:
            char = hiragana[i-1] + char
            result.pop()
            result.append(hiragana_vowel_map.get(char, ''))
        else:
            result.append(hiragana_vowel_map.get(char, ''))
        i += 1
    return ''.join(result)

def process_input(input_text):
    if re.search(r'[가-힣]', input_text):
        if invalid_detect(('0', '', input_text, '')):
            raise ValueError("유효하지 않은 한글 입력입니다.")
        processed = preprocess(input_text)
        if processed:
            return vowel_extraction(processed)
        else:
            raise ValueError("유효하지 않은 한글 입력입니다.")

    elif re.search(r'[ぁ-ん]', input_text):
        return convert_hiragana_vowel(input_text)

    else:
        raise ValueError("유효하지 않은 입력입니다. 한글 또는 히라가나를 입력해 주세요.")

def find_matching_indices(vowel_sequence):
    matching_indices = []
    first_priority_indices = []
    seen_lyrics = set()

    with open('tmp/vowel.txt', 'r', encoding='utf-8') as vowel_f:
        lines = vowel_f.readlines()

        for line in lines:
            parts = line.strip().split(' : ')
            if len(parts) == 2:
                index, vowels = parts
                if vowels not in seen_lyrics:
                    if vowel_sequence in vowels:
                        if vowels.endswith(vowel_sequence):
                            matching_indices.insert(0, index)
                            first_priority_indices.append(index)
                        else:
                            matching_indices.append(index)
                        seen_lyrics.add(vowels)

    return matching_indices, len(first_priority_indices)

user_input = input("라임을 찾고 싶은 구절을 한글 또는 히라가나로 입력해 주세요: ")

try:
    vowel_sequence = process_input(user_input)
    matching_indices, first_priority_count = find_matching_indices(vowel_sequence)

    print(f"총 {len(matching_indices)}개의 라임이 일치하는 구절이 발견되었습니다.")
    print(f"그 중 맨 끝이 일치하는 구절: {first_priority_count}개")

except ValueError as e:
    print(e)

라임을 찾고 싶은 구절을 한글 또는 히라가나로 입력해 주세요: あいうえお
총 285개의 라임이 일치하는 구절이 발견되었습니다.
그 중 맨 끝이 일치하는 구절: 42개


# 7. 일치하는 가사 블럭 출력
이 셀에서는 라임이 일치하는 가사 블럭을 `index.txt` 파일에서 읽어와 출력합니다.

1. **`display_lyrics_blocks` 함수**:
    - `index.txt` 파일을 읽어 각 인덱스에 해당하는 가사 블럭을 `index_to_block` 딕셔너리에 저장합니다.
    - 라임이 일치하는 인덱스 리스트를 받아 상위 50개의 가사 블럭을 순차적으로 출력합니다.
    - 각 가사 블럭은 원문, 발음, 번역 순으로 출력됩니다.
    - 일치하는 가사 블럭이 50개를 초과할 경우, 상위 50개만 출력하고 나머지는 생략되었음을 알립니다.

최종적으로 라임이 일치하는 가사 블럭의 내용을 사용자에게 표시합니다.


In [19]:
def display_lyrics_blocks(matching_indices):

    with index_file.open('r', encoding='utf-8') as index_f:
        lines = index_f.readlines()

        index_to_block = {}
        for i in range(0, len(lines), 4):
            if i + 3 < len(lines):
                index = lines[i].strip()
                jp_line = lines[i+1].strip()
                kr_pron_line = lines[i+2].strip()
                kr_trans_line = lines[i+3].strip()
                index_to_block[index] = (jp_line, kr_pron_line, kr_trans_line)

        for idx, index in enumerate(matching_indices[:50]):
            print(f"원문: {index_to_block[index][0]}")
            print(f"발음: {index_to_block[index][1]}")
            print(f"번역: {index_to_block[index][2]}")
            print()

        if len(matching_indices) > 50:
            print(f"총 {len(matching_indices)}개의 일치하는 가사 블럭 중 상위 50개만 표시했습니다. 나머지 {len(matching_indices) - 50}개는 생략되었습니다.")

display_lyrics_blocks(matching_indices)

원문: 悔しくても
발음: 쿠야시쿠테모
번역: 분하더라도

원문: 傷口はいつでも
발음: 키즈구치와 이츠데모
번역: 상처는 언제나

원문: つらい事が あればいつでも！
발음: 츠라이 코토가 아레바 이츠데모!
번역: 괴로운 일이 있다면 언제라도!

원문: 落ち込んでいる 時はいつでも！
발음: 오치콘데이루 토키와 이츠데모!
번역: 빠져들게 된다면 언제나!

원문: 捨てた願いも苦い夢も
발음: 스테타 네가이모 니가이 유메모
번역: 버렸던 소원도 씁쓸한 꿈도

원문: 「会いたい」が傷付く合図でも
발음: 아이타이가 키즈츠쿠 아이즈데모
번역: 「만나고 싶어」가 상처받는 신호더라도

원문: 結婚前は忙しくても
발음: 켓콘마에와 이소가시쿠테모
번역: 결혼 전에는 바쁘더라도

원문: 馬鹿でダメで報われなくて惨めたらしくても
발음: 바카데 다메데 무쿠와레나쿠테 미지메타라시쿠테모
번역: 바보라 안 돼, 보답받지 못해 비참해진다고 해도

원문: 「メリハリが大切」とは言うけど
발음: 메리하리가 타이세츠토와 이우케도
번역: 「치고빠짐이 중요」라고 말하지만은

원문: 寂しくなったらいつでも
발음: 사비시쿠 낫타라 이츠데모
번역: 외로워졌다면 언제든지

원문: 考え直したら悲しくても
발음: 칸가에나오시타라 카나시쿠테모
번역: 다시 생각했더니 슬퍼졌지만

원문: 眠る場所を求めるものだと　君は云うけど―。
발음: 네무루 바쇼오 모토메루 모노다토 키미와 이우케도.
번역: 잠들 장소를 원하는 거라고 너는 말했지만.

원문: 今が悲しくても
발음: 이마가 카나시쿠테모
번역: 지금이 슬프더라도

원문: この場所はいつでも
발음: 코노 바쇼와 이츠데모
번역: 이 장소는 언제나

원문: 根を伸ばす想いはいつでも
발음: 네오 노바스 오모이와 이츠데모
번역: 뿌리를 뻗는 마음은 언제나

원문: どれだけ　悲しくても
발음: 도레다케 카나시쿠테모
번역: 아무리 슬프더라도

원문: 夢をみていた　覚めること無い夢を
발음: 유메오 미테이타 사메루 코토 나이 유메오
번역: 꿈을 꾸고 있었어 깨어날 리 없는 꿈을

원문: ただ投げ出したい夢を
발음: 타